In [ ]:
%matplotlib ipympl
import twtof
import numpy as np
import matplotlib.pyplot as plt
from mpl_image_segmenter import ImageSegmenter
import ipywidgets as widgets


filename = "./data/20230722_magnetotactic_HO027_1_Au_pos_spot2_run1_30kV_50pA.h5"
filename = "./data/20241017_Fluorine_HO_134_1_spot8_run1_30kV_50pA.h5"
fib, peak = twtof.imread(filename)

# Define the maximum number of regions
N_ROI = 10

# Create the widgets
slice_selector = widgets.IntSlider(
    value=0, min=0, max=fib.shape[0] - 1, description="Slice"
)
class_selector = widgets.Dropdown(options=list(range(1, N_ROI + 1)), description="ROI")
erasing_button = widgets.Checkbox(value=False, description="Erasing")
refresh_btn = widgets.Button(value=False, description="Refresh")
vmax_slider = widgets.FloatSlider(
    value=0.5, min=0, max=1, step=0.001, description="Scale graph"
)


# Update function
def update(change):
    # update the active ROI/class
    multi_class_segmenter.current_class = class_selector.value
    # toggle to erase mode
    multi_class_segmenter.erasing = erasing_button.value
    # set the image plane in the left panel
    multi_class_segmenter._displayed.set_data(fib[slice_selector.value])

    # retreive the current mask and downscale it
    d = fib.shape[-1] // peak.shape[-1]
    lbl = multi_class_segmenter.mask[::d, ::d].astype(int)

    # for each label
    for c in np.unique(lbl)[1:]:
        # draw the averaged spectrum in the region
        f = peak.to_numpy()[:, slice_selector.value, lbl == c].mean(axis=1)
        axpeak[c - 1][0].set_data(peak["mass"].to_numpy(), 1e-3 + f)
    ax[1].set_ylim([0, np.pow(vmax_slider.value, 4.0)])
    refresh_btn.value = False


# Add callbacks
erasing_button.observe(update, names="value")
class_selector.observe(update, names="value")
slice_selector.observe(update, names="value")
refresh_btn.on_click(update)
fig, ax = plt.subplots(1, 2)
ax[0].set_axis_off()
ax[0].set_title("FIB Image")
axpeak = [ax[1].plot([0, 323], [0, 0]) for k in range(N_ROI)]
ax[1].set_box_aspect(1)
ax[1].set(
    xlim=(0, peak["mass"].to_numpy().max()),
    ylim=(0, 1),
    xlabel="mass/charge",
    ylabel="average ion/extraction",
    title="Mass spectrum",
)

# plt.tight_layout()
multi_class_segmenter = ImageSegmenter(
    fib[0], classes=N_ROI, mask_alpha=0.76, ax=ax[0], props={"color": "black"}
)

display(
    widgets.HBox(
        [
            widgets.VBox([slice_selector, vmax_slider]),
            widgets.VBox([class_selector, erasing_button]),
            refresh_btn,
        ]
    )
)

In [ ]:
import pandas as pd
from pathlib import Path

# Define the label for the FIB image
fib_labels = multi_class_segmenter.mask.astype(int)

# Resize the label to the size of the spectrum data
d = fib.shape[-1] // peak.shape[-1]
peak_labels = fib_labels[::d, ::d]

# Compute the average of the spectrums in each ROI
df = pd.concat(
    [
        pd.DataFrame(
            {
                "filename": Path(filename).stem,
                "mass": np.repeat(peak["mass"].to_numpy(), len(peak["Z"])),
                "slice": np.tile(peak["Z"].to_numpy(), len(peak["mass"])),
                "region": k,
                "mean": peak.to_numpy()[:, :, peak_labels == k].mean(axis=2).ravel(),
                # **{f'ROI {k}':peak.to_numpy()[:, :, peak_labels == k].mean(axis=2).ravel()
            }
        )
        for k in np.unique(peak_labels)
    ]
)

df


## Sub-slice selection
We might want to select only a few slices from the results and average over those slices.

In [ ]:
# create a new dataframe with only the selected slices
slice_start = 3
slice_stop = 5
selected_df = df[np.logical_and(df["slice"] >= slice_start, df["slice"] <= slice_stop)]

# compute the average of the spectrum per ROI over the slices
selected_mean_df = selected_df.groupby(['filename','region','mass'])[['slice','mean']].mean().reset_index()

# use a pivot table to reformat the table
selected_mean_df.pivot_table(values="mean",index="mass",columns=["filename","slice","region"])


In [ ]:
selected_mean_df.pivot_table(values="mean",index="mass",columns=["filename","slice","region"]).to_csv(f"{filename}-slices-{slice_start}-{slice_stop}.csv")

In [ ]:
import seaborn as sns
plt.figure()
sns.lineplot(selected_mean_df,x="mass",y="mean",hue="region")
plt.gca().set_ylim([0,0.05])